# Get images and data from cars.com for selected years

In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import cPickle
import urllib
import requests
import os

chromedriver = "/Users/Chris/Desktop/DSI/chromedriver"

### base search URL is:
https://www.cars.com/for-sale/searchresults.action?zc=92027&rd=75&prMn=5000&prMx=1000000&stkTypId=28881&yrId=58487&photoId=46724&sf1Nm=location&sf1Dir=ASC&sf2Nm=price&sf2Dir=DESC&page=1&perPage=100&sortFeatures=buryUsedLowPrice&sortFeatures=buryNewLowPrice&sortFeatures=buryLowPriceOlderThanSix&sortFeatures=buryNoPrice&searchSource=GN_REFINEMENT

### may need to allow scripts to run on the browser that pops up

## Basic outline of top-level scraping of cars.com for vehicle overview id numbers for later use

- search within 75 miles of zipcode
- Loops through each year between 2005:2012
- price: min 5000, max:1000000
- ad has photo
- return 100 listings per page


In [5]:
# function to collect listings for a given zipcode with the parameters listed above
# return a data frame and a directory full of images
def get_listings_by_zipcode(zc,year_id):

    # prepare the output df
    detail_cols = ['ad_id', 'bodyStyle', 'city', 'dealerName', 'doorCount', 'downloadDate', 'drivetrain',
               'engine', 'exteriorColor', 'fuelType', 'interiorColor', 'isGhostPhoto', 'isStockPhoto',
               'listingDate', 'location', 'makeName', 'miles', 'modelName', 'modelYear', 'mpgCity',
               'mpgHwy', 'oneOwner', 'photoUrlsLarge', 'price', 'privatePartyListing', 'seatCapacity',
               'sellerNotesPt1', 'sellerNotesPt2', 'sellerPhotoCount', 'state', 'transmission', 'trimName',
               'vin', 'zipcode']
    
    listings_df = pd.DataFrame(columns=detail_cols)

    
    # set the photo directory
    img_base = '/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom/carscom_images/'
    img_dir = img_base + str(zc)
    
    try:
        os.mkdir(img_dir)
    except:
        print img_dir,' already exists!'
    
    # count how many items are collected
    listing_count = 0
    image_count = 0
    
    # create empty lists to store the output
    makeName, modelName, modelYear, trimName = [],[],[],[]
    price, miles, exteriorColor, interiorColor = [],[],[],[]
    bodyStyle, doorCount, transmission, drivetrain = [],[],[],[]
    engine, fuelType,mpgCity,mpgHwy = [],[],[],[]
    city,dealerName,privatePartyListing,state = [],[],[],[]
    vin,zipcode,location,seatCapacity = [],[],[],[]
    listingDate,photoUrlsLarge,sellerPhotoCount = [],[],[]
    oneOwner,sellerNotesPt1,sellerNotesPt2 = [],[],[]
    isStockPhoto,isGhostPhoto,downloadDate,ad_id = [],[],[],[]
    
    
#     # get year ids so that the search can be broken down into smaller groups
#     year_id_string = '58487&yrId=56007&yrId=51683&yrId=47272&yrId=39723&yrId=34923&yrId=27381&yrId=20201&yrId=20145&yrId=20200&yrId=20144&yrId=20199'
#     years_ids = year_id_string.split('&yrId=')

#     open a new browser instance
#     browser = webdriver.Chrome(executable_path = chromedriver)

#     for page in pages_to_follow:
#     for year_id in years_ids:
        
    # create the page to follow
    pt1 = "https://www.cars.com/for-sale/searchresults.action?zc="
    pt2 = str(zc)
    pt3 = "&rd=75&prMn=5000&stkTypId=28881&yrId="
    pt4 = str(year_id)
    pt5 = "&photoId=46724&sf1Nm=price&sf1Dir=DESC&sf2Nm=miles&sf2Dir=ASC&page=" 
    pt6 = '1'
    # temp set to 10 per page for training, 100 for collecting
    pt7 = "&perPage=100&sortFeatures=buryUsedLowPrice&sortFeatures=buryNewLowPrice&sortFeatures=buryLowPriceOlderThanSix&sortFeatures=buryNoPrice&sortFeatures=buryUsedLowMileage&searchSource=GN_REFINEMENT"
    link = pt1 + pt2 +  pt3 +  pt4 + pt5 + pt6 + pt7

    # follow the link and get the html
    browser.get(link)
    time.sleep(9)
    HTML = browser.page_source
    soup = BeautifulSoup(HTML)

    # get the number of listings for the year from heading1
    # use this to determine how many pages to follow
    num_items = soup.h1.get_text().split(' ')[0].replace(',','')
    # add one for np.arange so that we'll get the last page
    num_pages = int(num_items)/100 + 1

    # make sure that there are no more than 50 pages, as that's the limit
    if num_pages > 50:
        num_pages = 50

    # make the list of pages for creating the links in the loop below
    pages = np.arange(1,num_pages,1)

    print 'there are %s listings over %s pages for year_id %s' % (num_items,num_pages,year_id)
    
    # limit pages for testing...
    #pages = [1]

    # iterate through each page and collect the ids
    for page in pages:
        pt6 = str(page)
        link = pt1 + pt2 +  pt3 +  pt4 + pt5 + pt6 + pt7

        print 'now parsing page %s at %s' % (page, str(time.ctime()))
        
        # empty list to store the overview_ids
        overview_list = []

        browser.get(link)
#         timeout = 10
#         try:
#             element_present = EC.presence_of_element_located((By.ID, 'h1'))
#             WebDriverWait(browser, timeout).until(element_present)
#         except TimeoutException:
#             print "Timed out waiting for page to load"

        time.sleep(9)
        HTML = browser.page_source
        soup = BeautifulSoup(HTML)

        # go through each link in the html and pull out the overview id
        # there are multiple occurences of each overview id, so will use
        # np.unique later to get the unique list
        for row in soup.find_all('a',href=True):
            try: 
                overview_id = row['href'].split('/')[3]
                if len(overview_id) == 9:
                    overview_list.append(overview_id)
            except:
                pass

        # only keep unique values in list
        overview_list = np.unique(overview_list)

        # extract the information for listings on each page
        for i,id_num in enumerate(overview_list):

            # prepare the links
            ajax_start = 'https://www.cars.com/ajax/listingsapi/1.0/listing/detail/'
            ajax_full = ajax_start + str(id_num)

            # use requests to get the listing
            # need to use try/except as some of them fail to load
            try:
                r = requests.get(ajax_full)
                # add a random time delay to keep from getting kicked off...
                sleep_time = np.random.randint(3,7)
                time.sleep(sleep_time)

            except:
                print 'unable to access %s at listing count %s' % (ajax_full,listing_count)
                time.sleep(10)
                pass
                
            # listing is in json format, and all details are in first dict
            try:
                overview = r.json()['listingDetailDto']
                listing_count += 1
                #print overview
            except:
                #print 'this one failed:',ajax_full
                #json_fail_count += 1
                pass
            
            if (listing_count % 250) == 0:
                print 'parsed %s listings' % listing_count


            # fill the lists
            # empty values are null
            makeName.append(overview['makeName'])
            modelName.append(overview['modelName'])
            modelYear.append(overview['modelYear'])
            trimName.append(overview['trimName'])
            price.append(overview['price'])
            miles.append(overview['milesInteger'])
            exteriorColor.append(overview['exteriorColor'])
            interiorColor.append(overview['interiorColor'])
            bodyStyle.append(overview['bodystyleName'])
            doorCount.append(overview['doorCount'])
            transmission.append(overview['transmission'])
            drivetrain.append(overview['drivetrain'])
            engine.append(overview['engineDescription'])
            fuelType.append(overview['fuelType'])
            mpgCity.append(overview['mpgCity'])
            mpgHwy.append(overview['mpgHwy'])
            city.append(overview['city'])
            dealerName.append(overview['dealerName'])
            privatePartyListing.append(overview['privatePartyListing'])
            state.append(overview['state'])
            vin.append(overview['vin'])
            zipcode.append(overview['zipcode'])
            location.append(overview['location'])
            seatCapacity.append(overview['seatCapacity'])
            listingDate.append(time.ctime(overview['listingDateSeconds']))
            photoUrlsLarge.append(overview['photoUrlsLarge'])
            sellerPhotoCount.append(overview['sellerPhotoCount'])
            oneOwner.append(overview['oneOwner'])
            sellerNotesPt1.append(overview['sellerNotesPart1'])
            sellerNotesPt2.append(overview['sellerNotesPart2'])
            isStockPhoto.append(overview['isStockPhoto'])
            isGhostPhoto.append(overview['isGhostPhoto'])
            ad_id.append(id_num)
            downloadDate.append(time.strftime("%m-%d-%Y", time.localtime()))


            # first check that the images will exist and be reasonable
            if (((len(overview['photoUrlsLarge'])) > 2) & (overview['isGhostPhoto'] == False) & (overview['isStockPhoto'] == False) & (overview['makeName'] != None) & (overview['modelName'] != None)):

                # now iterate through the list
                # limit to first 5 photos to minimize number of interior images
                # and to speed things up and save space
                for num,photo in enumerate(overview['photoUrlsLarge'][0:5]):

                    # create file name
                    photo_name = overview['makeName'].lower() + '_' + overview['modelName'].lower() + '_' + str(overview['modelYear'])+ '_' + str(id_num) + '_' + str(num) + '.jpg'

                    # full file name for photo with directory
                    photo_file = img_dir + '/' + photo_name

                    # see if it already exists
                    
                    if os.path.exists(photo_file):
                        pass
                    else:

                        try:
                            f = open(photo_file,'wb')
                            f.write(requests.get(photo).content)
                            f.close()
                            image_count += 1
                            if (image_count % 500) == 0:
                                print 'at image %s' % image_count

                        except:
                            pass

        # save the results from each page into a temp data frame

        # create a dataframe
        temp_df = pd.DataFrame({
        'makeName':makeName,
        'modelName':modelName,
        'modelYear':modelYear,
        'trimName':trimName,
        'price':price,
        'miles':miles,
        'exteriorColor':exteriorColor,
        'interiorColor':interiorColor,
        'bodyStyle':bodyStyle,
        'doorCount':doorCount,
        'transmission':transmission,
        'drivetrain':drivetrain,
        'engine':engine,
        'fuelType':fuelType,
        'mpgCity':mpgCity,
        'mpgHwy':mpgHwy,
        'city':city,
        'dealerName':dealerName,
        'privatePartyListing':privatePartyListing,
        'state':state,
        'vin':vin,
        'zipcode':zipcode,
        'location':location,
        'seatCapacity':seatCapacity,
        'listingDate':listingDate,
        'photoUrlsLarge':photoUrlsLarge,
        'sellerPhotoCount':sellerPhotoCount,
        'oneOwner':oneOwner,
        'sellerNotesPt1':sellerNotesPt1,
        'sellerNotesPt2':sellerNotesPt2,
        'isStockPhoto':isStockPhoto,
        'isGhostPhoto':isGhostPhoto,
        'downloadDate':downloadDate,
        'ad_id':ad_id
        #'photoBase':photo_base
        })

        # save it to file after every page just in case...
        f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom/carscom_listings/temp_listing_df.p','w')  # w = write, r = read
        cPickle.dump(temp_df,f)
        f.close()


    print "parsed year_id %s, %s records and %s photos collected" % (year_id,listing_count,image_count)

    # concatentate all the listings
    listings_df = pd.concat([listings_df,temp_df],axis=0)
    
    # close the browser
    #browser.quit()



    return listings_df  


In [7]:
# get year ids so that the search can be broken down into smaller groups
year_id_string = '58487&yrId=56007&yrId=51683&yrId=47272&yrId=39723&yrId=34923&yrId=27381&yrId=20201&yrId=20145&yrId=20200&yrId=20144&yrId=20199'
years_ids = year_id_string.split('&yrId=')

print years_ids[0:3]
print years_ids[4:7]
print years_ids[7:10]
print years_ids[10:]

['58487', '56007', '51683']
['39723', '34923', '27381']
['20201', '20145', '20200']
['20144', '20199']


In [8]:
# # open a new browser instance
# browser = webdriver.Chrome(executable_path = chromedriver)


# listings_table_92027_58487 = get_listings_by_zipcode(92027,58487)

# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_58487.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_58487,f)
# f.close()

# print listings_table_92027_58487.shape

/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_images/92027  already exists!
there are 2843 listings over 29 pages for year_id 58487
now parsing page 1 at Sun Sep 18 07:22:05 2016
now parsing page 2 at Sun Sep 18 07:35:30 2016
now parsing page 3 at Sun Sep 18 07:35:58 2016
at image 500
now parsing page 4 at Sun Sep 18 07:50:13 2016
at image 1000
parsed 250 listings
now parsing page 5 at Sun Sep 18 08:04:44 2016
at image 1500
now parsing page 6 at Sun Sep 18 08:18:54 2016
at image 2000
parsed 500 listings
now parsing page 7 at Sun Sep 18 08:32:17 2016
at image 2500
now parsing page 8 at Sun Sep 18 08:46:47 2016
at image 3000
now parsing page 9 at Sun Sep 18 09:01:07 2016
parsed 750 listings
at image 3500
now parsing page 10 at Sun Sep 18 09:15:19 2016
at image 4000
now parsing page 11 at Sun Sep 18 09:29:01 2016
parsed 1000 listings
now parsing page 12 at Sun Sep 18 09:43:25 2016
at image 4500
now parsing page 13 at Sun Sep 18 09:56:52 2016
at image 5000
now parsing page 14 

In [11]:
# # open a new browser instance
# browser = webdriver.Chrome(executable_path = chromedriver)

# # done!
# listings_table_92027_56007 = get_listings_by_zipcode(92027,56007)

# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_56007.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_56007,f)
# f.close()

# print listings_table_92027_56007.shape

/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_images/92027  already exists!
there are 5321 listings over 50 pages for year_id 56007
now parsing page 1 at Sun Sep 18 13:47:00 2016
now parsing page 2 at Sun Sep 18 14:01:28 2016
at image 500
now parsing page 3 at Sun Sep 18 14:16:13 2016
at image 1000
parsed 250 listings
now parsing page 4 at Sun Sep 18 14:31:12 2016
at image 1500
now parsing page 5 at Sun Sep 18 14:46:16 2016
at image 2000
parsed 500 listings
now parsing page 6 at Sun Sep 18 14:59:37 2016
at image 2500
now parsing page 7 at Sun Sep 18 15:14:01 2016
at image 3000
now parsing page 8 at Sun Sep 18 15:27:51 2016
parsed 750 listings
at image 3500
now parsing page 9 at Sun Sep 18 15:42:09 2016
at image 4000
now parsing page 10 at Sun Sep 18 15:56:28 2016
parsed 1000 listings
at image 4500
now parsing page 11 at Sun Sep 18 16:11:03 2016
now parsing page 12 at Sun Sep 18 16:25:40 2016
at image 5000
now parsing page 13 at Sun Sep 18 16:40:59 2016
parsed 1250 listings

In [4]:
# browser = webdriver.Chrome(executable_path = chromedriver)

# # had to split this one into two parts
# # first listings are saved in listings_table_92027_51683a.p,listings_table_92027_51683b.p

# # done!

# # reset the page to 38 before starting this one
# listings_table_92027_51683 = get_listings_by_zipcode(92027,51683)

# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_51683.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_51683,f)
# f.close()

# browser.quit()

# print listings_table_92027_51683.shape

/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_images/92027  already exists!
there are 5018 listings over 50 pages for year_id 51683
now parsing page 38 at Tue Sep 20 21:22:48 2016
now parsing page 39 at Tue Sep 20 21:38:07 2016
at image 500
now parsing page 40 at Tue Sep 20 21:53:48 2016
at image 1000
parsed 250 listings
now parsing page 41 at Tue Sep 20 22:08:53 2016
at image 1500
unable to access https://www.cars.com/ajax/listingsapi/1.0/listing/detail/677487841 at listing count 361
now parsing page 42 at Tue Sep 20 22:27:48 2016
at image 2000
parsed 500 listings
now parsing page 43 at Tue Sep 20 22:42:05 2016
at image 2500
now parsing page 44 at Tue Sep 20 22:55:18 2016
now parsing page 45 at Tue Sep 20 23:08:46 2016
at image 3000
parsed 750 listings
now parsing page 46 at Tue Sep 20 23:21:41 2016
at image 3500
now parsing page 47 at Tue Sep 20 23:34:56 2016
at image 4000
parsed 1000 listings
now parsing page 48 at Tue Sep 20 23:48:02 2016
at image 4500
now parsing page

In [13]:
# browser = webdriver.Chrome(executable_path = chromedriver)

# listings_table_92027_39723 = get_listings_by_zipcode(92027,39723)

# done for pages 1-14
# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_39723.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_39723,f)
# f.close()

# browser.quit()

# print listings_table_92027_39723.shape

/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_images/92027  already exists!
there are 2543 listings over 26 pages for year_id 39723
now parsing page 1 at Tue Sep 20 11:00:02 2016
now parsing page 2 at Tue Sep 20 11:11:54 2016
at image 500
now parsing page 3 at Tue Sep 20 11:23:55 2016
at image 1000
parsed 250 listings
now parsing page 4 at Tue Sep 20 11:36:16 2016
at image 1500
now parsing page 5 at Tue Sep 20 11:48:59 2016
at image 2000
parsed 500 listings
now parsing page 6 at Tue Sep 20 12:01:31 2016
at image 2500
now parsing page 7 at Tue Sep 20 12:15:50 2016
at image 3000
now parsing page 8 at Tue Sep 20 12:27:31 2016
parsed 750 listings
at image 3500
now parsing page 9 at Tue Sep 20 12:39:39 2016
at image 4000
now parsing page 10 at Tue Sep 20 12:52:09 2016
at image 4500
parsed 1000 listings
now parsing page 11 at Tue Sep 20 13:04:47 2016
at image 5000
now parsing page 12 at Tue Sep 20 13:16:50 2016
now parsing page 13 at Tue Sep 20 13:18:18 2016
at image 5500
now pa

In [15]:
# browser = webdriver.Chrome(executable_path = chromedriver)

# # repeat for pages 15-end
# listings_table_92027_39723 = get_listings_by_zipcode(92027,39723)

# # done!
# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_39723.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_39723,f)
# f.close()

# browser.quit()

# # print listings_table_92027_39723.shape

/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_images/92027  already exists!
there are 2551 listings over 26 pages for year_id 39723
now parsing page 15 at Tue Sep 20 14:03:51 2016
now parsing page 16 at Tue Sep 20 14:12:32 2016
now parsing page 17 at Tue Sep 20 14:25:27 2016
at image 500
parsed 250 listings
now parsing page 18 at Tue Sep 20 14:38:02 2016
at image 1000
now parsing page 19 at Tue Sep 20 14:51:24 2016
at image 1500
parsed 500 listings
now parsing page 20 at Tue Sep 20 15:03:28 2016
at image 2000
now parsing page 21 at Tue Sep 20 15:16:18 2016
at image 2500
now parsing page 22 at Tue Sep 20 15:29:21 2016
parsed 750 listings
at image 3000
now parsing page 23 at Tue Sep 20 15:46:16 2016
at image 3500
now parsing page 24 at Tue Sep 20 15:58:27 2016
at image 4000
parsed 1000 listings
now parsing page 25 at Tue Sep 20 16:10:43 2016
at image 4500
parsed year_id 39723, 1126 records and 4678 photos collected


In [ ]:
# browser = webdriver.Chrome(executable_path = chromedriver)
# listings_table_92027_34923 = get_listings_by_zipcode(92027,34923)

# # done!

# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_34923.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_34923,f)
# f.close()

# browser.quit()
# print listings_table_92027_34923.shape

/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_images/92027  already exists!
there are 1765 listings over 18 pages for year_id 34923
now parsing page 1 at Tue Sep 20 16:40:28 2016
now parsing page 2 at Tue Sep 20 16:48:48 2016
now parsing page 3 at Tue Sep 20 16:58:22 2016
unable to access https://www.cars.com/ajax/listingsapi/1.0/listing/detail/672678850 at listing count 229
unable to access https://www.cars.com/ajax/listingsapi/1.0/listing/detail/673045795 at listing count 233
parsed 250 listings
now parsing page 4 at Tue Sep 20 18:06:03 2016
now parsing page 5 at Tue Sep 20 18:06:24 2016
now parsing page 6 at Tue Sep 20 18:19:55 2016
at image 500
parsed 500 listings
now parsing page 7 at Tue Sep 20 18:34:33 2016
at image 1000
now parsing page 8 at Tue Sep 20 18:50:11 2016
at image 1500
now parsing page 9 at Tue Sep 20 19:05:41 2016
parsed 750 listings
at image 2000
now parsing page 10 at Tue Sep 20 19:21:24 2016
at image 2500
now parsing page 11 at Tue Sep 20 19:37:03 20

In [5]:
# browser = webdriver.Chrome(executable_path = chromedriver)

# #done!
# listings_table_92027_27381 = get_listings_by_zipcode(92027,27381)


# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_27381.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_27381,f)
# f.close()

# print listings_table_92027_27381.shape

In [6]:

# # done!
# listings_table_92027_20201 = get_listings_by_zipcode(92027,20201)

# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_20201.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_20201,f)
# f.close()

# print listings_table_92027_20201.shape
# browser.quit()

In [28]:
# browser = webdriver.Chrome(executable_path = chromedriver)

# #done

# listings_table_92027_20145 = get_listings_by_zipcode(92027,20145)

# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_20145.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_20145,f)
# f.close()

# browser.quit()

# print listings_table_92027_20145.shape

/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_images/92027  already exists!
there are 1350 listings over 14 pages for year_id 20145
now parsing page 1 at Mon Sep 19 14:17:29 2016
now parsing page 2 at Mon Sep 19 14:29:22 2016
at image 500
now parsing page 3 at Mon Sep 19 14:41:07 2016
at image 1000
parsed 250 listings
now parsing page 4 at Mon Sep 19 14:53:18 2016
at image 1500
now parsing page 5 at Mon Sep 19 15:04:49 2016
at image 2000
parsed 500 listings
now parsing page 6 at Mon Sep 19 15:17:07 2016
now parsing page 7 at Mon Sep 19 15:17:29 2016
now parsing page 8 at Mon Sep 19 15:17:46 2016
now parsing page 9 at Mon Sep 19 15:18:02 2016
at image 2500
now parsing page 10 at Mon Sep 19 15:30:08 2016
at image 3000
now parsing page 11 at Mon Sep 19 15:42:22 2016
parsed 750 listings
at image 3500
now parsing page 12 at Mon Sep 19 15:55:00 2016
now parsing page 13 at Mon Sep 19 15:55:20 2016
parsed year_id 20145, 822 records and 3718 photos collected
(823, 34)


In [29]:
# browser = webdriver.Chrome(executable_path = chromedriver)

# listings_table_92027_20200 = get_listings_by_zipcode(92027,20200)

# # done!
# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_20200.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_20200,f)
# f.close()

# browser.quit()

# print listings_table_92027_20200.shape

/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_images/92027  already exists!
there are 1262 listings over 13 pages for year_id 20200
now parsing page 1 at Mon Sep 19 16:07:49 2016
now parsing page 2 at Mon Sep 19 16:20:09 2016
at image 500
now parsing page 3 at Mon Sep 19 16:32:20 2016
at image 1000
parsed 250 listings
now parsing page 4 at Mon Sep 19 16:45:07 2016
at image 1500
now parsing page 5 at Mon Sep 19 16:57:48 2016
at image 2000
parsed 500 listings
now parsing page 6 at Mon Sep 19 17:23:58 2016
at image 2500
now parsing page 7 at Mon Sep 19 17:36:09 2016
at image 3000
now parsing page 8 at Mon Sep 19 17:50:25 2016
now parsing page 9 at Mon Sep 19 17:50:44 2016
parsed 750 listings
at image 3500
now parsing page 10 at Mon Sep 19 18:03:31 2016
at image 4000
now parsing page 11 at Mon Sep 19 18:16:24 2016
unable to access https://www.cars.com/ajax/listingsapi/1.0/listing/detail/674607224 at listing count 962
at image 4500
parsed 1000 listings
now parsing page 12 at Mo

In [25]:
# browser = webdriver.Chrome(executable_path = chromedriver)

# # done!
# listings_table_92027_20144 = get_listings_by_zipcode(92027,20144)

# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_20144.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_20144,f)
# f.close()

# browser.quit()

/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_images/92027  already exists!
there are 1023 listings over 11 pages for year_id 20144
now parsing page 1 at Mon Sep 19 12:16:12 2016
now parsing page 2 at Mon Sep 19 12:28:03 2016
at image 500
now parsing page 3 at Mon Sep 19 12:40:40 2016
now parsing page 4 at Mon Sep 19 12:41:03 2016
now parsing page 5 at Mon Sep 19 12:41:20 2016
at image 1000
parsed 250 listings
now parsing page 6 at Mon Sep 19 12:57:28 2016
at image 1500
now parsing page 7 at Mon Sep 19 13:15:15 2016
at image 2000
parsed 500 listings
now parsing page 8 at Mon Sep 19 13:28:01 2016
at image 2500
now parsing page 9 at Mon Sep 19 13:45:11 2016
at image 3000
now parsing page 10 at Mon Sep 19 13:57:51 2016
parsed 750 listings
at image 3500
parsed year_id 20144, 821 records and 3788 photos collected
/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_images/92027  already exists!
there are 734 listings over 8 pages for year_id 20199
now parsing page 1 at Mo

KeyboardInterrupt: 

In [4]:
# browser = webdriver.Chrome(executable_path = chromedriver)

# # done!
# listings_table_92027_20199 = get_listings_by_zipcode(92027,20199)

# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom_listings/listings_table_92027_20199.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_20199,f)
# f.close()

# browser.quit()


/Applications/anaconda/envs/dsi/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /Applications/anaconda/envs/dsi/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


there are 731 listings over 8 pages for year_id 20199
now parsing page 1 at Fri Sep 16 13:47:59 2016
at image 500
now parsing page 2 at Fri Sep 16 14:03:29 2016
at image 1000
at image 1500
now parsing page 3 at Fri Sep 16 14:19:33 2016
at image 2000
parsed 250 listings
at image 2500
now parsing page 4 at Fri Sep 16 14:35:35 2016
at image 3000
at image 3500
now parsing page 5 at Fri Sep 16 14:51:20 2016
at image 4000
parsed 500 listings
at image 4500
now parsing page 6 at Fri Sep 16 15:08:05 2016
at image 5000
now parsing page 7 at Fri Sep 16 15:24:47 2016
parsed year_id 20199, 618 records and 5436 photos collected


In [6]:
# # forgot to collect year 2013!!
# # 47272
# # 92027

# browser = webdriver.Chrome(executable_path = chromedriver)

# # done!
# listings_table_92027_47272 = get_listings_by_zipcode(92027,47272)

# # save the processed list
# f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom/carscom_listings/listings_table_92027_47272.p','w')  # w = write, r = read
# cPickle.dump(listings_table_92027_47272,f)
# f.close()

# #browser.quit()


/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom/carscom_images/92027  already exists!
there are 5575 listings over 50 pages for year_id 47272
now parsing page 1 at Wed Sep 21 20:51:42 2016
now parsing page 2 at Wed Sep 21 21:05:40 2016
at image 500
now parsing page 3 at Wed Sep 21 21:18:29 2016
at image 1000
parsed 250 listings
now parsing page 4 at Wed Sep 21 21:31:49 2016
at image 1500
now parsing page 5 at Wed Sep 21 21:44:39 2016
at image 2000
parsed 500 listings
now parsing page 6 at Wed Sep 21 21:58:19 2016
at image 2500
now parsing page 7 at Wed Sep 21 22:12:31 2016
at image 3000
now parsing page 8 at Wed Sep 21 22:25:50 2016
parsed 750 listings
at image 3500
now parsing page 9 at Wed Sep 21 22:38:06 2016
at image 4000
now parsing page 10 at Wed Sep 21 22:52:20 2016
parsed 1000 listings
at image 4500
now parsing page 11 at Wed Sep 21 23:04:57 2016
now parsing page 12 at Wed Sep 21 23:17:36 2016
at image 5000
now parsing page 13 at Wed Sep 21 23:30:16 2016
parsed 1250 

KeyboardInterrupt: 

In [8]:
# forgot to collect year 2013!!
# 47272
# 94066

browser = webdriver.Chrome(executable_path = chromedriver)

# done!
listings_table_94066_47272 = get_listings_by_zipcode(94066,47272)

# save the processed list
f = open('/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom/carscom_listings/listings_table_94066_47272.p','w')  # w = write, r = read
cPickle.dump(listings_table_94066_47272,f)
f.close()

browser.quit()


/Users/Chris/Desktop/DSI-SF-2-clrife/capstone/data/carscom/carscom_images/94066  already exists!
there are 5409 listings over 50 pages for year_id 47272
now parsing page 1 at Thu Sep 22 09:01:13 2016
now parsing page 2 at Thu Sep 22 09:13:24 2016
at image 500
now parsing page 3 at Thu Sep 22 09:24:45 2016
at image 1000
parsed 250 listings
now parsing page 4 at Thu Sep 22 09:36:31 2016
at image 1500
now parsing page 5 at Thu Sep 22 09:48:26 2016
at image 2000
now parsing page 6 at Thu Sep 22 09:59:31 2016
at image 2500
now parsing page 7 at Thu Sep 22 10:09:29 2016
at image 3000
now parsing page 8 at Thu Sep 22 10:18:46 2016
at image 3500
now parsing page 9 at Thu Sep 22 10:27:58 2016
at image 4000
now parsing page 10 at Thu Sep 22 10:36:45 2016
at image 4500
now parsing page 11 at Thu Sep 22 10:45:45 2016
at image 5000
now parsing page 12 at Thu Sep 22 10:54:53 2016
parsed 500 listings
at image 5500
now parsing page 13 at Thu Sep 22 11:06:53 2016
at image 6000
now parsing page 14 at Th